Load necessary packages

In [2]:
import pandas as pd
import numpy as np


Data loading and inspection

In [6]:
df = pd.read_json("../data/trec-medline.json", lines=True)


In [9]:
print(df.columns)
df.head(5)

Index(['index', 'AB', 'AD', 'CY', 'DA', 'DCOM', 'DP', 'EDAT', 'ID', 'IP', 'IS',
       'JID', 'LA', 'MHDA', 'PG', 'PMID', 'PST', 'PT', 'SB', 'SO', 'TA', 'TI',
       'UI', 'VI', 'MH', 'RN', 'FAU', 'LR', 'SI', 'RF', 'PS', 'EIN', 'CN',
       'AID', 'PHST', 'TT', 'CI', 'CON', 'CIN', 'RPF', 'RPI', 'SPIN', 'RIN',
       'ROF', 'ORI', 'UOF', 'UIN'],
      dtype='object')


,index,AB,AD,CY,DA,DCOM,DP,EDAT,ID,IP,...,CON,CIN,RPF,RPI,SPIN,RIN,ROF,ORI,UOF,UIN
0,{'_id': '1'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,We present an evaluation of the accuracy and p...,Department of Molecular Biology and Skaggs Ins...,Netherlands,20011105.0,20020401.0,2001 Sep,2001/11/06 10:00,GM56879/GM/NIGMS,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,{'_id': '2'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,An analysis is presented of experimental versu...,"Department of Medical Biosciences, Medical Bio...",Netherlands,20011105.0,20020401.0,2001 Sep,2001/11/06 10:00,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,{'_id': '3'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
id_rows = df.iloc[::2].reset_index(drop=True)[["index"]]
id_rows["index"] = id_rows["index"].apply(lambda x: x["_id"])
content_rows = df.iloc[1::2].reset_index(drop=True).drop(labels=["index"], axis=1)
combined_df = pd.concat([id_rows, content_rows], axis=1)

combined_df.head()

,index,AB,AD,CY,DA,DCOM,DP,EDAT,ID,IP,...,CON,CIN,RPF,RPI,SPIN,RIN,ROF,ORI,UOF,UIN
0,1,We present an evaluation of the accuracy and p...,Department of Molecular Biology and Skaggs Ins...,Netherlands,20011105.0,20020401.0,2001 Sep,2001/11/06 10:00,GM56879/GM/NIGMS,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,An analysis is presented of experimental versu...,"Department of Medical Biosciences, Medical Bio...",Netherlands,20011105.0,20020401.0,2001 Sep,2001/11/06 10:00,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,The global fold of maltose binding protein in ...,Protein Engineering Network Center of Excellen...,Netherlands,20011105.0,20020401.0,2001 Sep,2001/11/06 10:00,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,A general method is presented for magnetic fie...,"Molecular Structure Division, National Institu...",Netherlands,20011105.0,20020401.0,2001 Sep,2001/11/06 10:00,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,The dependence between the anomeric carbon che...,"Department of Chemistry & Biochemistry, Univer...",Netherlands,20011105.0,20020401.0,2001 Sep,2001/11/06 10:00,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
docs = combined_df[["index", "AB"]]
docs.head(5)

,index,AB
0,1,We present an evaluation of the accuracy and p...
1,2,An analysis is presented of experimental versu...
2,3,The global fold of maltose binding protein in ...
3,4,A general method is presented for magnetic fie...
4,5,The dependence between the anomeric carbon che...


In [35]:
# load queries
queries = pd.DataFrame(columns=["index", "query"])

with open("../data/training-queries-simple.txt", "r") as f:
    lines = f.readlines()

data = []
for line in lines:
    x = line.strip().split("\t")
    if len(x) >= 2:  
        data.append({"index": x[0], "query": x[1]})
    else:
        raise ValueError("wtf")
queries = pd.concat([queries, pd.DataFrame(data)], ignore_index=True)
print(queries)

   index                                              query
0      1  "cyclin-dependent kinase inhibitor 1A (p21, Ci...
1      2  "DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide ...
2      3  ets variant gene 6 (TEL oncogene) in Homo sapiens
3      4  fibroblast growth factor 7 (keratinocyte growt...
4      5  "glycine receptor, alpha 1 (startle disease/hy...
5      6  "major histocompatibility complex, class II, D...
6      7  Janus kinase 2 (a protein tyrosine kinase) in ...
7      8  luteinizing hormone/choriogonadotropin recepto...
8      9  metallothionein 3 (growth inhibitory factor (n...
9     10  protein C (inactivator of coagulation factors ...
10    11  "ras-related C3 botulinum toxin substrate 1 (r...
11    12              tropomyosin 1 (alpha) in Homo sapiens
12    13    frizzled homolog 4 (Drosophila) in Homo sapiens
13    14            tyrosyl-tRNA synthetase in Homo sapiens
14    15                major vault protein in Homo sapiens
15    16    "adrenergic receptor, alpha 